In [2]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import ultralytics
ultralytics.__version__

'8.3.83'

In [28]:
import os
print(os.listdir(r"C:\Users\USER\Documents\MLProjects\VehicleSpeedDetection\sort"))

['.git', '.gitignore', 'data', 'LICENSE', 'README.md', 'requirements.txt', 'sort.py']


In [31]:
import sys
sys.path.append(r"C:\Users\USER\Documents\MLProjects\VehicleSpeedDetection\sort")

from sort import*


In [ ]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO




In [5]:
# Import the model
model = YOLO("model/yolov8s.pt")

In [6]:
class_list = ["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","traffic light"]

In [34]:
tracker = cv2.TrackerCSRT_create()
count = 0

AttributeError: module 'cv2' has no attribute 'TrackerCSRT_create'

In [8]:
print(tracker)

In [13]:
# Import video
cap = cv2.VideoCapture("video/highway_mini.mp4")

In [14]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    # Perform object detection with YOLO
    results = model.predict(frame)

    # Parse results
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    
    # Prepare detections for DeepSort
    detections = []
    embeddings = []  # ✅ Keep as a Python list

    for _, row in px.iterrows():
        x1, y1, x2, y2 = map(int, row[:4])  # Bounding box coordinates
        confidence = float(row[4])
        class_id = int(row[5])
        detected_class = class_list[class_id]

        # Filter for specific vehicle classes
        if detected_class in ["car", "truck", "motorcycle"]:
            detections.append(([x1, y1, x2, y2], confidence))

            # Placeholder for extracting embeddings (if using DeepSORT with embeddings)
            embedding = np.zeros((128,))  # Replace with real embeddings if available
            embeddings.append(embedding)  # ✅ Fix: Append correctly to the list

    # Convert embeddings to NumPy array safely
    embeddings = np.array(embeddings) if embeddings else np.zeros((len(detections), 128))

    # Ensure embeddings length matches detections
    if len(detections) != len(embeddings):
        print(f"Warning: Mismatch! Detections: {len(detections)}, Embeddings: {len(embeddings)}")
        embeddings = np.zeros((len(detections), 128))

    # Ensure valid embeddings
    embeddings = np.nan_to_num(embeddings)

    # Pass detections to tracker
    if len(detections) > 0:
        bbox_id = tracker.update_tracks(detections, embeds=embeddings if len(embeddings) > 0 else None, frame=frame)

        # Process the bounding boxes and assign IDs
        for track in bbox_id:
            x3, y3, x4, y4 = track.to_tlbr()  # Get bounding box
            track_id = track.track_id  # Get the track ID
            
            cx, cy = (x3 + x4) // 2, (y3 + y4) // 2  # Compute center
            
            # Draw bounding box, center point, and ID
            cv2.circle(frame, (int(cx), int(cy)), 4, (0, 0, 255), -1)
            cv2.rectangle(frame, (int(x3), int(y3)), (int(x4), int(y4)), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (int(x3), int(y3) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Show the frame
    cv2.imshow("frames", frame)

    # Exit on 'Esc' key press
    if cv2.waitKey(0) & 0xFF == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 320x640 9 cars, 3 trucks, 124.0ms
Speed: 2.7ms preprocess, 124.0ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)


c:\Users\USER\Documents\MLProjects\VehicleSpeedDetection\venv\Lib\site-packages\deep_sort_realtime\deep_sort\nn_matching.py:52: RuntimeWarning: invalid value encountered in divide
  a = np.asarray(a) / np.linalg.norm(a, axis=1, keepdims=True)
c:\Users\USER\Documents\MLProjects\VehicleSpeedDetection\venv\Lib\site-packages\deep_sort_realtime\deep_sort\nn_matching.py:53: RuntimeWarning: invalid value encountered in divide
  b = np.asarray(b) / np.linalg.norm(b, axis=1, keepdims=True)


ValueError: matrix contains invalid numeric entries